# 📌 Assigning WCS Using Gaia Catalog and Image Star Detection

When processing lots of data it would become extremely tiresome to need to look up star coordinates to anchor to for each image. This tutorial demonstrates how to:
- Detect stars in astronomical images automatically 
- Query Gaia DR3 catalog for known star positions
- Match pixel coordinates to sky coordinates
- Compute and assign WCS headers using Astropy
- Process a **folder of FITS images** using reusable functions

In [2]:
# Required packages
import numpy as np
from astropy.io import fits
from astropy.wcs.utils import fit_wcs_from_points
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.nddata import NDData
from astroquery.gaia import Gaia
from photutils.detection import DAOStarFinder
from astropy.stats import sigma_clipped_stats
import matplotlib.pyplot as plt
import os, glob

## 🔧 Function: Detect Stars in Image

In [4]:
def detect_stars(image_data, fwhm=3.0, threshold_sigma=5.0):
    mean, median, std = sigma_clipped_stats(image_data, sigma=3.0)
    daofind = DAOStarFinder(fwhm=fwhm, threshold=threshold_sigma*std)
    sources = daofind(image_data - median)
    return sources

## 🌌 Function: Query Gaia Around a Given RA/Dec

In [6]:
def query_gaia(ra, dec, radius_arcmin=5):
    coord = SkyCoord(ra*u.deg, dec*u.deg)
    job = Gaia.cone_search_async(coord, radius=radius_arcmin*u.arcmin)
    return job.get_results()

## 🔭 Function: Assign WCS to Image Based on Matches

In [8]:
def assign_wcs(image_path, center_ra, center_dec):
    with fits.open(image_path) as hdul:
        data = hdul[0].data
        header = hdul[0].header
        shape = data.shape

    sources = detect_stars(data)
    xy_coords = np.array([sources['xcentroid'], sources['ycentroid']]).T
    gaia_results = query_gaia(center_ra, center_dec)
    sky_coords = SkyCoord(ra=gaia_results['ra']*u.deg, dec=gaia_results['dec']*u.deg)

    # Limit to same number of sources for rough matching (this can be improved)
    n = min(len(sky_coords), len(xy_coords))
    wcs = fit_wcs_from_points(xy=xy_coords[:n], world_coords=sky_coords[:n], image_shape=shape)
    
    header.update(wcs.to_header())
    fits.writeto(image_path.replace(".fits", "_wcs.fits"), data, header, overwrite=True)

## 📁 Function: Process a Folder of FITS Files

In [14]:
def batch_assign_wcs(folder_path, ra_center, dec_center):
    fits_files = sorted(glob.glob(os.path.join(folder_path, '*.fits')))
    for path in fits_files:
        print(f"Processing {path}...")
        assign_wcs(path, ra_center, dec_center)
    print("Done.")

## ▶️ Run on Your Folder of Data
Make sure your images cover the same sky region and are centered approximately at the same RA/Dec.

In [ ]:
# Example: run on folder with images near RA=150.0, Dec=2.0
# batch_assign_wcs('path/to/fits_folder', 150.0, 2.0)